<a href="https://colab.research.google.com/github/GitMishka/1/blob/main/MIMO_to_CRD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from datetime import datetime, timedelta
from google.colab import file
# Load the Excel file, skipping the first 15 rows
file_path_moveout = 'rs_SeniorMoveOut (2)(1).xlsx'  # Adjust this path to match the location of your uploaded file
df_moveout = pd.read_excel(file_path_moveout, skiprows=15)

# Initial population of 'Community' column based on the first column's "Community:" markers
df_moveout['Community'] = None
current_community = None
for index, row in df_moveout.iterrows():
    if str(row.iloc[0]).startswith('Community:'):
        current_community = row.iloc[0].split(':', 1)[1].strip()
    else:
        df_moveout.at[index, 'Community'] = current_community

# Now refine 'Community' column values by applying LEFT(RIGHT(community,5),4) logic
df_moveout['Community'] = df_moveout['Community'].apply(lambda x: x[-5:-1] if x is not None else None)

# Continue with the rest of the processing
df_moveout = df_moveout.dropna(subset=['Resident '])
df_moveout['Move Out\nDate'] = pd.to_datetime(df_moveout['Move Out\nDate'], errors='coerce')
values_to_delete = [
    "Morning Pointe of Athens (aths)", "Death-At Community", "Death-Outside Community", "Financial-Competitor", "Move Out \nReason", "Move-out Analysis \nby Move Out Reason",
    "Financial-Family Home", "Health improved", "Higher LOC-Nursing Home", "Relocating to other area",
    "Morning Pointe of Brentwood (brwd)", "Discharged to Family", "Evicted-Financial", "Higher LOC-Hospice",
    "Higher LOC-Memory Care",
    "Morning Pointe of Calhoun (calh)", "Dissatisfied-Competitor", "Evicted-Behavior", "Evicted-Medical",
    "Morning Pointe of Chattanooga Shallowford (chtt)", "Dissatisfied-Family Home", "Financial-Own Home",
    "Transfer to Sister MP",
    "Morning Pointe of Clinton (clin)", "Higher LOC-Geri/Psych",
    "Morning Pointe of Columbia (colm)",
    "Morning Pointe of Danville (danv)",
    "Morning Pointe of East Hamilton (eham)", "Dissatisfied-Own Home",
    "Morning Pointe of Frankfort (frkt)",
    "Morning Pointe of Franklin (frln)",
    "Morning Pointe of Franklin TN (fktn)",
    "Morning Pointe of Greenbriar (grnb)",
    "Morning Pointe of Greeneville (grnv)",
    "Morning Pointe of Happy Valley (hppy)",
    "Morning Pointe of Hardin Valley (hard)", "Natural Disaster - Moved to Competitor",
    "Morning Pointe of Hixson (hixn)",
    "Morning Pointe of Knoxville (knox)",
    "Morning Pointe of Lenoir City (lenc)",
    "Morning Pointe of Lexington (lexn)", "End of Respite",
    "Morning Pointe of Lexington-East (lexe)",
    "Morning Pointe of Louisville (lvlm)",
    "Morning Pointe of Powell (powl)",
    "Morning Pointe of Richmond (rich)",
    "Morning Pointe of Russell (russ)",
    "Morning Pointe of Spring Hill (sprh)",
    "Morning Pointe of Tullahoma (tula)",
    "Morning Pointe of Tuscaloosa (tusc)",
    "The Lantern at Morning Pointe of Chattanooga (chtl)",
    "The Lantern at Morning Pointe of Collegedale (cgdl)",
    "The Lantern at Morning Pointe of Frankfort (frkl)",
    "The Lantern at Morning Pointe of Franklin TN (fktl)",
    "The Lantern at Morning Pointe of Knoxville (knxl)",
    "The Lantern at Morning Pointe of Lenoir City (lenl)",
    "The Lantern at Morning Pointe of Lexington (lexl)",
    "The Lantern at Morning Pointe of Louisville (lvll)",
    "The Lantern at Morning Pointe of Powell (pwll)",
    "The Lantern at Morning Pointe of Russell (rusl)",
    "The Lantern at Morning Pointe of Spring Hill (sprl)"
]
# Filter out rows where 'Resident ' column contains any of the specified values
df_moveout = df_moveout[~df_moveout['Resident '].isin(values_to_delete)]

# Defer deletion of specified columns until after all other processing
df_moveout = df_moveout.loc[:, ~df_moveout.columns.str.contains('^Unnamed')]
df_moveout['ID'] = df_moveout['Resident '].apply(lambda x: x[x.find("("):x.find(")")+1])

# Extract the name part for the 'Name' column
df_moveout['Name'] = df_moveout['Resident '].apply(lambda x: '('.join(x.split('(')[:-1]).strip())

# Extract Move Out Month name from 'Move Out\nDate'
df_moveout['Move Out Month'] = df_moveout['Move Out\nDate'].dt.month_name()

# For "Community/Campus", assuming 'Community' is already processed as per your instructions
df_moveout['Community/Campus'] = df_moveout['Community']

# Now select only the columns you need
df_moveout_final = df_moveout[['ID', 'Name', 'Move Out Month', 'Community/Campus']]
# This will align rows based on the 'ID' column and update/add the columns from df_moveout_final into df_moveout
df_combined = pd.merge(df_moveout, df_moveout_final, on='ID', how='left')


# Save the processed DataFrame to a new Excel file
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
output_filename = f'moveout_summary_{timestamp}.xlsx'
#df_moveout_final.to_excel(output_filename, index=False)
df_moveout.to_excel(output_filename, index=False)

files.download(output_filename)

<ipython-input-2-fd162493b6c3>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_moveout['Move Out\nDate'] = pd.to_datetime(df_moveout['Move Out\nDate'], errors='coerce')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Install openpyxl if not already installed
!pip install openpyxl

import openpyxl

# Load the workbook
wb = openpyxl.load_workbook('rs_SeniorMoveIn.xlsx')

# Delete the sheets "Document map" and "Sheet3" if they exist
sheets_to_delete = ['Document map', 'Sheet3']
for sheet in sheets_to_delete:
    if sheet in wb.sheetnames:
        std = wb[sheet]
        wb.remove(std)

# Delete the first 11 rows in "Sheet2", preserving headers in row 12
if 'Sheet2' in wb.sheetnames:
    ws = wb['Sheet2']
    for row in range(1, 12):
        ws.delete_rows(1)

# Save the workbook with changes
wb.save('modified_rs_SeniorMoveIn.xlsx')
import pandas as pd
from datetime import datetime, timedelta

# Load the data
file_path = 'modified_rs_SeniorMoveIn.xlsx'  # Adjust this to your file's path
df_movein = pd.read_csv(file_path)

# Your existing data processing steps...

# Convert dates from string to datetime
df_movein['Service Start\nDate'] = pd.to_datetime(df_movein['Service Start\nDate'], errors='coerce')
df_movein['Move In\nDate'] = pd.to_datetime(df_movein['Move In\nDate'], errors='coerce')

# Assume 'server_start_date_str' is the server start date in 'YYYY-MM-DD' format
server_start_date_str = '2024-03-01'  # Example server start date
server_start_date = datetime.strptime(server_start_date_str, '%Y-%m-%d')

# Calculate the date 3 months before the server start date
three_months_prior = server_start_date - timedelta(days=90)

# Filter the DataFrame to include only rows where the 'Service Start\nDate' is within the last three months before the server start date
df_movein = df_movein[(df_movein['Service Start\nDate'] >= three_months_prior) & (df_movein['Service Start\nDate'] < server_start_date)]

# Continue with your existing data processing steps...

# Remove the first 10 rows of the DataFrame
df_movein = df_movein.iloc[10:].reset_index(drop=True)

# Create a timestamp for the filename
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
df_movein = df_movein.loc[:, ~df_movein.columns.str.contains('Unnamed')]  # NEW LINE TO DELETE UNNAMED COLUMNS

# Save the cleaned and updated dataframe to a new CSV with timestamp in filename
filename = f'movein_summary_{timestamp}.csv'
df_movein.to_csv(filename, index=False)

# Display the filename for confirmation
#print(f"File saved as: {filename}")

# Count the number of entries per region and display the counts
# weights_sum_by_region = df_movein.groupby('Region')['Weights'].sum()
# print(weights_sum_by_region)


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc1 in position 16: invalid start byte

In [ ]:
#!pip install openpyxl

import openpyxl
from copy import copy
# Load the workbook
wb = openpyxl.load_workbook('rs_SeniorMoveIn.xlsx')

# Delete the sheets "Document map" and "Sheet3" if they exist
sheets_to_delete = ['Document map', 'Sheet3']
for sheet in sheets_to_delete:
    if sheet in wb.sheetnames:
        std = wb[sheet]
        wb.remove(std)

if 'Sheet2' in wb.sheetnames:
    ws = wb['Sheet2']

    # First, let's identify any merged cells that need to be handled.
    merged_ranges = list(ws.merged_cells.ranges)
    for merged_range in merged_ranges:
        ws.unmerge_cells(str(merged_range))

    # We'll copy the data and formatting for rows starting from 12 upwards
    for row in range(12, ws.max_row + 1):
        for col in range(1, ws.max_column + 1):
            src_cell = ws.cell(row=row, column=col)
            dest_cell = ws.cell(row=row-11, column=col)

            # Copy value
            dest_cell.value = src_cell.value

            # Copy style
            if src_cell.has_style:
                dest_cell.font = copy(src_cell.font)
                dest_cell.border = copy(src_cell.border)
                dest_cell.fill = copy(src_cell.fill)
                dest_cell.number_format = copy(src_cell.number_format)
                dest_cell.protection = copy(src_cell.protection)
                dest_cell.alignment = copy(src_cell.alignment)

    # Delete the now-empty rows at the bottom
    ws.delete_rows(2, 11)

    # Re-merge cells as needed, adjusting for the shifted content
    for range in merged_ranges:
        new_start_row = max(1, range.min_row - 11)
        new_end_row = max(1, range.max_row - 11)
        ws.merge_cells(start_row=new_start_row, start_column=range.min_col, end_row=new_end_row, end_column=range.max_col)

# Save the workbook with changes
wb.save('corrected_rs_SeniorMoveIn.xlsx')

In [ ]:
wb = openpyxl.load_workbook('rs_SeniorMoveIn.xlsx')

# Ensure "Sheet2" is present


NameError: name 'copy' is not defined